In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr

# --------------------------
# 配置参数（与你的回测规则对齐）
# --------------------------
CONFIG = {
    "factortable_path": r'./factortable.parquet',
    "selection_result_path": r'./short_term_selection_optimized.csv',
    "buy_delay": 1,    # 与你的回测一致：T+1买入
    "sell_delay": 5,   # 与你的回测一致：T+5卖出
    "ic_ir_output_path": r'./ic_ir_aligned_backtest.csv'
}

# --------------------------
# 1. 加载基础数据（含完整行情，用于匹配买卖价）
# --------------------------
def load_base_data():
    df = pd.read_parquet(CONFIG["factortable_path"])
    # 数据预处理：保留原始股票代码格式（不转小写，与选股结果对齐）
    df["date"] = pd.to_datetime(df["date"])  # 转为datetime类型
    # 按股票和日期排序，添加交易序列（用于匹配延迟后的价格）
    df = df.sort_values(by=["stock_code", "date"]).reset_index(drop=True)
    df["trade_seq"] = df.groupby("stock_code").cumcount()  # 每个股票的交易日期序列
    # 保留核心列
    base_df = df[["stock_code", "date", "trade_seq", "close"]].dropna(subset=["close"])
    # 数据校验
    print(f"基础数据加载完成：")
    print(f"- 总记录数：{len(base_df)}条")
    print(f"- 股票数量：{base_df['stock_code'].nunique()}只")
    print(f"- 日期范围：{base_df['date'].min()} ~ {base_df['date'].max()}")
    print(f"- 股票代码示例：{base_df['stock_code'].iloc[0]}")
    return base_df

# --------------------------
# 2. 加载因子数据（修复格式匹配问题）
# --------------------------
def load_factor_data(base_df):
    # 加载选股结果
    factor_df = pd.read_csv(CONFIG["selection_result_path"])
    # 数据预处理：关键修复——与基础数据格式完全对齐
    # 修复1：股票代码保留原始格式（不转小写）
    # 修复2：日期转为datetime类型（与base_df的date格式一致）
    factor_df["date"] = pd.to_datetime(factor_df["date"])
    # 修复3：从base_df获取trade_seq（避免重复计算导致的序列不一致）
    base_seq = base_df[["stock_code", "date", "trade_seq"]].copy()
    # 合并trade_seq（按股票代码+日期精确匹配）
    factor_df = pd.merge(
        factor_df,
        base_seq,
        on=["stock_code", "date"],
        how="left",
        validate="one_to_one"  # 确保一对一匹配，避免重复
    )
    # 数据清洗：过滤无trade_seq的记录（匹配失败的记录）
    factor_df = factor_df.dropna(subset=["trade_seq", "total_score"])
    factor_df["trade_seq"] = factor_df["trade_seq"].astype(int)  # 确保序列为整数
    # 保留必要列
    factor_df = factor_df[["stock_code", "date", "trade_seq", "total_score"]].rename(
        columns={"total_score": "factor_value", "date": "select_date"}
    )
    # 数据校验
    print(f"\n因子数据加载完成：")
    print(f"- 总记录数：{len(factor_df)}条")
    print(f"- 匹配失败记录数：{len(pd.read_csv(CONFIG['selection_result_path'])) - len(factor_df)}条")
    if len(factor_df) > 0:
        print(f"- 股票代码示例：{factor_df['stock_code'].iloc[0]}")
        print(f"- 日期示例：{factor_df['select_date'].iloc[0]}")
        print(f"- 因子值示例：{factor_df['factor_value'].iloc[0]:.2f}")
    else:
        print("⚠️  因子数据加载失败！请检查以下内容：")
        print("1. 选股结果与factortable的股票代码格式是否一致（如是否都为SH600000/SZ000001）")
        print("2. 选股结果的date是否在factortable的date范围内")
    return factor_df

# --------------------------
# 3. 匹配买卖价，计算回测口径的收益率
# --------------------------
def calculate_backtest_returns(base_df, factor_df):
    if len(factor_df) == 0:
        return pd.DataFrame()  # 无数据时返回空表
    # 构建买卖序列：T日选股 → T+buy_delay买入 → T+sell_delay卖出
    factor_df["buy_seq"] = factor_df["trade_seq"] + CONFIG["buy_delay"]
    factor_df["sell_seq"] = factor_df["trade_seq"] + CONFIG["sell_delay"]
    
    # 构建价格映射表（股票+序列 → 价格）：用字典映射，效率更高
    price_map = base_df.set_index(["stock_code", "trade_seq"])["close"].to_dict()
    
    # 匹配买入价和卖出价
    def get_price(stock_code, seq):
        return price_map.get((stock_code, seq), np.nan)
    factor_df["buy_price"] = factor_df.apply(lambda x: get_price(x["stock_code"], x["buy_seq"]), axis=1)
    factor_df["sell_price"] = factor_df.apply(lambda x: get_price(x["stock_code"], x["sell_seq"]), axis=1)
    
    # 计算回测口径收益率（与你的代码完全一致）
    factor_df["return_rate"] = (factor_df["sell_price"] - factor_df["buy_price"]) / \
                              factor_df["buy_price"].replace(0, 0.0001) * 100  # 百分比收益率
    # 过滤有效收益记录
    valid_df = factor_df.dropna(subset=["buy_price", "sell_price", "return_rate"])
    # 数据校验
    print(f"\n收益率计算完成：")
    print(f"- 有效记录数：{len(valid_df)}条")
    print(f"- 无效记录数（无买卖价）：{len(factor_df) - len(valid_df)}条")
    if len(valid_df) > 0:
        print(f"- 平均收益率：{valid_df['return_rate'].mean():.2f}%（与回测结果对比）")
    return valid_df

# --------------------------
# 4. 计算适配回测规则的IC/IR（增加异常处理）
# --------------------------
def calculate_ic_ir(valid_df):
    if len(valid_df) == 0:
        raise ValueError("无有效数据用于IC计算，请先解决数据加载问题")
    
    # 按选股日（select_date）分组计算每日IC
    def calc_daily_ic(group):
        # 避免单条记录无法计算相关系数
        if len(group) < 2:
            return pd.Series({"IC": np.nan, "P_Value": np.nan, "Trade_Count": len(group)})
        # 因子值与回测收益率的Spearman相关系数
        ic, p_value = spearmanr(group["factor_value"], group["return_rate"])
        return pd.Series({"IC": ic, "P_Value": p_value, "Trade_Count": len(group)})
    
    # 执行每日IC计算：消除警告，过滤无效分组
    daily_ic = valid_df.groupby("select_date", observed=True).apply(
        calc_daily_ic, include_groups=False  # include_groups=False消除FutureWarning
    ).reset_index()
    daily_ic = daily_ic.dropna(subset=["IC"]).sort_values(by="select_date")
    
    # 计算核心指标
    ic_mean = daily_ic["IC"].mean()
    ic_std = daily_ic["IC"].std()
    ir = ic_mean / ic_std if ic_std != 0 else 0
    significant_days = len(daily_ic[daily_ic["P_Value"] < 0.05])
    
    # 输出结果
    print("\n" + "="*60)
    print(f"IC/IR结果（适配你的回测规则：T+1买 → T+5卖）")
    print("="*60)
    print(f"平均IC值：{ic_mean:.4f}")
    print(f"IC标准差：{ic_std:.4f}")
    print(f"信息比率IR：{ir:.4f}")
    print(f"有效计算日数：{len(daily_ic)}天")
    print(f"IC显著天数（P<0.05）：{significant_days}/{len(daily_ic)}")
    print(f"每日平均有效交易数：{daily_ic['Trade_Count'].mean():.0f}")
    print("="*60)
    
    # 保存结果
    daily_ic.to_csv(CONFIG["ic_ir_output_path"], index=False, encoding="utf-8-sig")
    print(f"\nIC/IR结果已保存至：{CONFIG['ic_ir_output_path']}")
    return daily_ic, ic_mean, ir

# --------------------------
# 主流程（增加异常捕获和引导）
# --------------------------
def main():
    try:
        # 1. 加载基础数据
        base_df = load_base_data()
        # 2. 加载因子数据（传入base_df确保格式一致）
        factor_df = load_factor_data(base_df)
        if len(factor_df) == 0:
            print("\n❌ 程序终止：无有效因子数据，请根据上述提示检查数据格式")
            return
        # 3. 计算回测口径收益率
        valid_df = calculate_backtest_returns(base_df, factor_df)
        if len(valid_df) == 0:
            print("\n❌ 程序终止：无有效收益率数据，请检查买卖序列是否超出行情数据范围")
            return
        # 4. 计算IC/IR
        daily_ic, ic_mean, ir = calculate_ic_ir(valid_df)
    except Exception as e:
        print(f"\n❌ 计算失败：{str(e)}")
        # 针对原报错“['IC']”的特殊处理
        if "'IC'" in str(e):
            print("⚠️  提示：该错误通常是因为每日分组内交易数过少（<2条），无法计算相关系数")
            print("建议：检查选股结果是否每日有足够多的交易记录（至少2条）")

if __name__ == "__main__":
    main()

基础数据加载完成：3345866条记录，5276只股票
因子数据加载完成：0条记录
收益率计算完成：0条有效记录（与回测有效交易数一致）

计算失败：['IC']
